Квадратная решётка. В каждой ячейке с вероятностью $p$ находится частица. Частицы образуют кластеры (учитываются связи только по горизонтали или вертикали). Если есть путь, соединяющий верхний ряд с нижним, то такой кластер называем бесконечным.

Задача: определить вероятность появления бесконечного кластера, то есть значение $p$, при котором бесконечный кластер появляется. $P(p) - ?$ Для каждого $p$ нужно провести много испытаний и вычислить вероятность появления бесконечного кластера. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
N = 4 # size of lattice
lattice = np.zeros((N, N))

def generate(probability):
    return np.random.choice([0, 1], p=[1-probability, probability])

In [3]:
def lattice_with_its_clusters(probability, N = 10):

    clusters = []
    lattice = np.zeros((N, N))

    def insert_node(x_cond, y_cond, x, y):
        for k in range(len(clusters)):
            if((x_cond, y_cond) in clusters[k]):
                clusters[k].add((x, y))
                break

    for i in range(N):
        for j in range(N):
            lattice[i, j] = generate(probability)

            if(lattice[i, j] == 1): #begin to check neighbours and app
                if(j == 0 and i == 0): #First case when i = j = 0
                    clusters.append(set())
                    clusters[0].add((i, j))
                elif(j > 0 and i == 0): #Second case when i = 0, j > 0
                    if (lattice[i, j-1] == 0):
                        clusters.append(set())
                        clusters[-1].add((i, j))
                    else:
                        insert_node(i, j-1, i, j)
                elif(j == 0 and i > 0): #Third case when i > 0, j = 0
                    if(lattice[i-1, j] == 0):
                        clusters.append(set())
                        clusters[-1].add((i, j))
                    else:
                        insert_node(i-1, j, i, j)
                else: #Last case when i > 0, j > 0
                    if(lattice[i, j-1] == 0 and lattice[i-1, j] == 0): #First case when 0, 1, 0 (left, (i,j), upper)
                        clusters.append(set())
                        clusters[-1].add((i, j))
                    elif(lattice[i, j-1] == 1 and lattice[i-1, j] == 0): #Second case when 1, 1, 0
                        insert_node(i, j-1, i, j)
                    elif(lattice[i, j-1] == 0 and lattice[i-1, j] == 1): #Third case when 0, 1, 1
                        insert_node(i-1, j, i, j)
                    else: #Fourth case when 1, 1, 1 splitted in 2 subcases
                        for k in range(len(clusters)):
                            if((i, j-1) in clusters[k]):
                                clusters[k].add((i, j)) #When neighbours are not in one cluster then:
                                if((i-1, j) not in clusters[k]): 
                                    clusters[k].add((i, j))
                                    for kk in range(len(clusters)): #merge sets via (i,j) node
                                        if((i-1, j) in clusters[kk]):
                                            for item in clusters[k]:
                                                clusters[kk].add(item)
                                            del clusters[k]
                                            break
                                    break
    return lattice, clusters

In [4]:
def has_infinite_cluster(clusters, N=10):
    for cluster in clusters:
        if(len(cluster) >= 10): #size less than number of nodes in length cannot create infinite cluster
            first = 0
            last = 0
            for node in cluster:
                if(node[0] == 0):
                    first = 1
                if(node[0] == N-1):
                    last = 1
            if(first == 1 and last == 1):
                return True
    return False

In [5]:
lat, clut = lattice_with_its_clusters(probability=0.48, N=10)
has_infinite_cluster(clut, N=10)

False

In [6]:
print(lat)
clut

[[1. 1. 1. 0. 0. 0. 1. 1. 1. 1.]
 [0. 1. 1. 0. 0. 1. 1. 1. 0. 1.]
 [0. 1. 1. 0. 1. 0. 0. 0. 1. 0.]
 [1. 0. 1. 1. 0. 1. 1. 0. 1. 1.]
 [0. 0. 1. 1. 1. 1. 0. 1. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 1. 0. 1. 0. 0. 1. 0. 1.]
 [0. 1. 1. 0. 0. 0. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0. 1. 0. 1. 0. 1.]]


[{(0, 6), (0, 7), (0, 8), (0, 9), (1, 5), (1, 6), (1, 7), (1, 9)},
 {(2, 4)},
 {(2, 8), (3, 8), (3, 9), (4, 7), (4, 8), (5, 7), (5, 8), (5, 9)},
 {(3, 0)},
 {(0, 0),
  (0, 1),
  (0, 2),
  (1, 1),
  (1, 2),
  (2, 1),
  (2, 2),
  (3, 2),
  (3, 3),
  (3, 5),
  (3, 6),
  (4, 2),
  (4, 3),
  (4, 4),
  (4, 5)},
 {(5, 1),
  (6, 0),
  (6, 1),
  (6, 2),
  (7, 1),
  (7, 2),
  (8, 1),
  (8, 2),
  (9, 0),
  (9, 1),
  (9, 2),
  (9, 3)},
 {(6, 4), (6, 5), (7, 4)},
 {(7, 7), (8, 6), (8, 7), (9, 7)},
 {(7, 9)},
 {(9, 5)},
 {(9, 9)}]